In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
xl = pd.ExcelFile("./raw_data/NWM_data_updated.xlsx", engine='openpyxl')
a=xl.sheet_names


sheet_no = list(range(2, 32))

In [2]:
#loading the data set
dataframe_collection = {}
i=0
for i in range(33):
    if i==0 :
        dataframe_collection[i] = xl.parse(a[i],converters={"STAID_1":str})
    else:
        dataframe_collection[i] = xl.parse(a[i])



In [3]:

x_mean = pd.read_csv("./raw_data/x_mean.csv", sep='\t')
x_std =  pd.read_csv("./raw_data/x_std.csv", sep='\t')
y_mean =  pd.read_csv("./raw_data/y_mean.csv", sep='\t')
y_std =  pd.read_csv("./raw_data/y_std.csv", sep='\t')

In [4]:
from keras.models import load_model
import statistics
import math
pred = [None] * 30
std = [None] * 30
std_err = [None] *30
obsflow = dataframe_collection[2]
for l in range(0,30) :
     
    model = load_model('./temporal_DLmodels/my_model_%d_day.h5'%(l))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    modelflow= dataframe_collection[l+3]
    temp =  [0] * 385
    for index, row in dataframe_collection[0].iterrows():
        column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
        df = pd.DataFrame(columns = column_names)  
        k=0
        d=0
      
        sta_id = dataframe_collection[0].iloc[index,0]
        dis2sea=dataframe_collection[0].iloc[index,1]
        land_use=dataframe_collection[0].iloc[index,2]
        soil_texture1=dataframe_collection[0].iloc[index,3]
        soil_texture2=dataframe_collection[0].iloc[index,4]
        soil_texture3=dataframe_collection[0].iloc[index,5]
        soil_texture4=dataframe_collection[0].iloc[index,6]
        soil_texture5=dataframe_collection[0].iloc[index,7]
        soil_texture6=dataframe_collection[0].iloc[index,8]
        soil_texture7=dataframe_collection[0].iloc[index,9]
        soil_texture8=dataframe_collection[0].iloc[index,10]
        soil_texture9=dataframe_collection[0].iloc[index,11]
        soil_texture10=dataframe_collection[0].iloc[index,12]
        topoindex = dataframe_collection[0].iloc[index,13]
        drainagearea =  dataframe_collection[0].iloc[index,14]
        obs = obsflow[sta_id]/drainagearea
        mod = modelflow[sta_id]/drainagearea

        for i in range(480,979):
            df.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
            k=k+1
        df =df.dropna()
            
        if df.empty:
            temp[index] = 0
            d=d+1
            print(index)
        else :
            x, y = df.iloc[ :, 0:15], df.iloc[:,15]
            x-= x_mean.iloc[l,1:16]
            x /= x_std.iloc[l,1:16]
            
            z=model.predict(x)
            
            x=x*x_std.iloc[l,1:16]
            x=x +x_mean.iloc[l,1:16]
            
            z-= y_mean.iloc[l,1]
            z /= y_std.iloc[l,1]
            
            z=z.ravel()
            z=abs(z)
            a=x.iloc[:,14]
            b=a-(2*z)
            c=a+(z*2)
            res=y.between(b,c)
            count = np.count_nonzero(res)
            temp[index] = count/len(y)   
       
    std[l]= statistics.stdev(temp)    
    pred[l] = sum(temp)/(385-d) 
    std_err[l] = (std[l]*2)/math.sqrt(385-d)
    del model


53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294
53
119
162
171
172
294


In [5]:
d_x=pd.DataFrame(pred)
d_y=pd.DataFrame(std)
d_z=pd.DataFrame(std_err)
d_xy=pd.concat([d_x, d_y, d_z], axis=1)
d_xy.columns = ['Pred', 'Std', 'Std_err']
d_xy
d_xy.to_csv("./graph_data/prediction_std_stderr_probability_ungagued.csv")

In [6]:
pred

[0.7438969289983491,
 0.7204898918989158,
 0.7228538781087596,
 0.7171034217019949,
 0.7062997558139581,
 0.7136892352513897,
 0.7117340448136001,
 0.7123155117887996,
 0.7100921559153643,
 0.7057779164661493,
 0.7059171501584022,
 0.7014820527467379,
 0.7071744117969314,
 0.7100013083074194,
 0.7104503514173931,
 0.7124249717730547,
 0.7167343620475455,
 0.7053393116677521,
 0.7079058501588537,
 0.7132750424382588,
 0.7184455254726144,
 0.7110748477929848,
 0.7048795818217549,
 0.7216477565418874,
 0.7186658719774746,
 0.7099499469749591,
 0.7199885882406188,
 0.7095936615380697,
 0.6969436835403324,
 0.7120991122262614]